In [13]:
from mastodon import Mastodon, StreamListener
import unicodedata
import re
import json
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import emoji

In [2]:
m = Mastodon(
    api_base_url=f'https://mastodon.au',
    access_token='bt1UoHZyDR5P5MS4jeMEXRog6QFcZJ3ZxSxtktlM-z0'
)

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name = 'cardiffnlp/twitter-roberta-base-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

d:\python38\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [22]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# tweet = "I like using the new transformers library!"

def find_sentiment(tweet):
    tweet=preprocess(tweet)
    inputs = tokenizer(tweet, return_tensors='pt',truncation=True,max_length=512).to(device)
    outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=-1)
    sentiment = ['Negative', 'Neutral', 'Positive']
    result = sentiment[probs.argmax().item()]

    max_prob, max_index=torch.max(probs,dim=1)

    return result,max_prob.item()


In [27]:
def contains_emoji(text):
    '''
    This function checks if a string text contains emoji element
    :param text: A string of row context of Toot
    :return: A boolean indicating if any emoji is found
    '''
    return any(is_emoji(c) for c in text)


def is_emoji(char):
    '''
    This function checks for if a character is emoji
    :param char: the target character
    :return: A boolean indicating if this character is emoji
    '''
    return unicodedata.category(char).startswith('So')


class Listener(StreamListener):
    def on_update(self, status):
        file = open("myfile.json", "a", encoding='utf-8')  # append mode
        obj = json.dumps(status, indent=0, sort_keys=True, default=str, ensure_ascii=False, separators=(',', ':'))
        obj = obj.replace("\n", "")

        obj =  re.sub("<[^>]*>", '', obj)
        if contains_emoji(obj):
            try:
                data = json.loads(obj)
                info = dict()
                text = data['content']
                if contains_emoji(text):
                    info['id'] = data['id']
                    info['author'] = data['account']['id']
                    info['account'] = data['account']
                    info['text'] = text
                    info['created_at'] = data['created_at']
                    sentiment, prob = find_sentiment(text)
                    info['sentiment'] = sentiment
                    info['sentiment_prob'] = prob
                    info['emoji'] = emoji.distinct_emoji_list(text)
                    file.write(json.dumps(info, indent=4) + '\n')
            except:
                pass



In [28]:
m.stream_public(Listener())

KeyboardInterrupt: 